## Import Libraries

In [24]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn

## Read Data

In [25]:
axial_image = 'MRNet-v1.0/train/axial/0000.npy'
coronal_image = 'MRNet-v1.0/train/coronal/0000.npy'
saggital_image = 'MRNet-v1.0/train/sagittal/0000.npy'


img_array_axial = np.load(axial_image)
img_array_coronal = np.load(coronal_image)
img_array_sagittal = np.load(saggital_image)

In [26]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'


In [27]:
def load_labels(abnormal_path, acl_path, meniscus_path):
    
    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])
    
    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))
    
    return df_lables_combined

In [28]:
# Load Train Labels
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,0000,1,0,0
1,0001,1,1,1
2,0002,1,0,0
3,0003,1,0,1
4,0004,1,0,0
...,...,...,...,...
1125,1125,1,0,1
1126,1126,1,0,1
1127,1127,0,0,0
1128,1128,1,0,0


In [29]:
# Load Valid Labels
df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [30]:
glob.glob('MRNet-v1.0\*')

['MRNet-v1.0\\train',
 'MRNet-v1.0\\train-abnormal.csv',
 'MRNet-v1.0\\train-acl.csv',
 'MRNet-v1.0\\train-meniscus.csv',
 'MRNet-v1.0\\valid',
 'MRNet-v1.0\\valid-abnormal.csv',
 'MRNet-v1.0\\valid-acl.csv',
 'MRNet-v1.0\\valid-meniscus.csv']

In [31]:
# Read Traning Data
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []

    axial_labels = []
    coronal_labels = []
    sagittal_labels = []

    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')
    
    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)
        
        for i in train_data:
#             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                axial_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'coronal' in i:
                coronal_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                coronal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])   
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                sagittal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])

        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images, axial_labels

train_path =  'MRNet-v1.0\\train'
valid_path =  'MRNet-v1.0\\valid'

axial_images_train, coronal_images_train, sagittal_images_train, axial_labels_train = read_data(train_path, df_labels_train)

Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\axial
Loading Complete From :    MRNet-v1.0\train\axial



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\coronal
Loading Complete From :    MRNet-v1.0\train\coronal



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\sagittal
Loading Complete From :    MRNet-v1.0\train\sagittal





In [32]:
# print(np.array(axial_images_train).shape)
print(np.array(coronal_images_train).shape)
print(np.array(sagittal_images_train).shape)
print(np.array(axial_labels_train).shape)

(1130, 256, 256, 16)
(1130, 256, 256, 16)
(1130, 3)


## Modeling

In [33]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


In [34]:
# Two Input Model

def model ():
    visible1 = Input(shape=(256,256,16))
    
    conv11 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(visible1)
    pool11 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv11)
    
    conv12 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv12)
    
    conv13 = Conv2D(filters=16, kernel_size=(5,5), activation='relu')(pool12)
    pool13 = MaxPooling2D(pool_size=(2, 2),strides=(2,2))(conv13)
    flat1 = Flatten()(pool13)
    
    return flat1, visible1
model()

# merge input models
model_1, Visible_1 = model()
model_2, Visible_2 = model()
model_3, Visible_3 = model()

merge = concatenate([model_1, model_2])

hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(5, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
model = Model(inputs=[Visible_1, Visible_2, Visible_3], outputs=output)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_7 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_15 (Conv2D)             (None, 252, 252, 16  6416        ['input_6[0][0]']                
                                )                                                           

In [46]:
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD( learning_rate = 0.000001, momentum = 0.9),metrics=["binary_accuracy"])

# Give Class Weights to Balance Results
class_weight = {0: 1.,
                1: 50.,
                2: 50.}

model.fit(x = [np.array(coronal_images_train), np.array(sagittal_images_train), np.array(axial_images_train)],
          y = np.array(axial_labels_train),
          epochs=5,
          batch_size=32,
          validation_split=0.20,
          class_weight=class_weight)

# model.save_weights('model_3_Input.h5')

NameError: name 'coronal_images_train' is not defined

In [36]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16110126214400142078
xla_global_id: -1
]


In [37]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [38]:
del coronal_images_train, sagittal_images_train

In [39]:
axial_images_valid, coronal_images_valid, sagittal_images_valid, axial_labels_valid = read_data(valid_path, df_labels_valid)

Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\axial
Loading Complete From :    MRNet-v1.0\valid\axial



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\coronal
Loading Complete From :    MRNet-v1.0\valid\coronal



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\sagittal
Loading Complete From :    MRNet-v1.0\valid\sagittal





## Evaluate Model

In [40]:
y_pred = model.predict([ np.array(coronal_images_valid), np.array(sagittal_images_valid), np.array(axial_images_valid)])

4/4 [==============================] - 2s 357ms/step


In [41]:
y_pred

array([[5.2456468e-01, 7.5321954e-01, 6.8978924e-01],
       [6.4483929e-01, 1.1578601e-01, 8.9546645e-01],
       [7.8981036e-01, 1.0855497e-02, 9.9156749e-01],
       [7.7058297e-01, 9.9999893e-01, 9.9994707e-01],
       [5.4716849e-01, 3.4411350e-01, 6.6406029e-01],
       [5.2845716e-01, 4.0410176e-01, 6.0116529e-01],
       [9.9759793e-01, 1.1889564e-09, 1.0000000e+00],
       [8.1453270e-01, 6.4082406e-03, 9.9517357e-01],
       [7.6969457e-01, 1.6100051e-02, 9.8719645e-01],
       [9.1790831e-01, 2.6670206e-04, 9.9983245e-01],
       [9.5427269e-01, 3.1799671e-05, 9.9998230e-01],
       [5.6944972e-01, 9.5979983e-01, 9.0655786e-01],
       [7.1621549e-01, 4.0874727e-02, 9.6557182e-01],
       [7.7157897e-01, 1.5534407e-02, 9.8767275e-01],
       [5.0606430e-01, 5.6837344e-01, 5.4911244e-01],
       [6.4130861e-01, 9.9863273e-01, 9.9118239e-01],
       [5.7448339e-01, 9.6789682e-01, 9.1978353e-01],
       [5.6351602e-01, 2.9511148e-01, 7.1501523e-01],
       [5.1434553e-01, 6.572

In [42]:
from sklearn.metrics import classification_report
# 
print(classification_report(y_pred.round(), axial_labels_valid))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88       120
           1       0.48      0.52      0.50        50
           2       0.98      0.44      0.60       117

   micro avg       0.86      0.60      0.70       287
   macro avg       0.82      0.58      0.66       287
weighted avg       0.90      0.60      0.70       287
 samples avg       0.69      0.61      0.62       287



C:\Users\Jackson\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
from sklearn.metrics import roc_auc_score
ypreds_abnormal = list(y_pred[:,0])
ytrue_abnormal = list(np.array(axial_labels_valid)[:,0])
auc_abnormal = roc_auc_score(y_score= ypreds_abnormal, y_true= ytrue_abnormal)

ypreds_acl = list(y_pred[:,0])
ytrue_acl = list(np.array(axial_labels_valid)[:,0])
auc_acl = roc_auc_score(y_score= ypreds_acl, y_true= ytrue_acl)


ypreds_meniscus = list(y_pred[:,0])
ytrue_meniscus = list(np.array(axial_labels_valid)[:,0])
auc_meniscus = roc_auc_score(y_score= ypreds_meniscus, y_true= ytrue_meniscus)


print(
    "Abnormal MRI AUC: " + str(auc_abnormal) + 
    "\nACL Injury AUC: " + str(auc_acl) +
    "\nMeniscus Injury AUC: " + str(auc_meniscus)
)


Abnormal MRI AUC: 0.5789473684210525
ACL Injury AUC: 0.5789473684210525
Meniscus Injury AUC: 0.5789473684210525


In [44]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

import torch
print(torch.cuda.is_available())

False
False
True
